<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pendiente:**
* calcular el promedio en C
* Obtener números aleatorios para cargar el vector


#1 Introducción

La aceleración es la tasa de variación de la velocidad de un objeto cuando se mueve. Si el objeto en cuestión mantiene su velocidad de forma constante, entonces no está acelerando, algo que sí hará cuando su velocidad cambia. La aceleración se mide en metros por segundo al cuadrado, y lo hace en base al tiempo que le lleva pasar de una velocidad a otra, o bien en una fuerza que se aplica sobre el objeto. [1]

Se puede calcular la aceleración de un cuerpo mediante la siguiente fórmula:

## <center>$\frac{vf-vi}{tf-ti} = {a}$

A continuación, calcularemos la aceleración de varios cuerpos utilizando OpenMP [5].

#2 Armado del ambiente



##2.1  Generar code_acel.cpp

Mediante código Python, genero una cadena de texto con código C. Luego, gurdo la cadena en el archivo code_acel.cpp

In [ ]:
code = """

#include <iostream>
#include <vector>
#include <cstdlib>
#include  <sys/time.h>
#include <omp.h> //Cabecera OpenMP


//------------------------------------------------
// Macros para medir el tiempo

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
  gettimeofday(&tv, NULL);    \
  dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
  
#define TIEMPO_FIN( h )      \
  gettimeofday(&tv, NULL);    \
  dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; //

#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL     1
#define HTH_ACEL_SEC  2
#define HTH_ACEL_OMP  3

//---------------------------------------------------------

int main(int argc, char*argv[])
{
  int i,c;
  TIEMPO_INI ( HTH_TOTAL )

  //Leo los parametros.
  if (argc != 4 )
  {
    std::cerr<<"Error al ingresar los parámetros" <<std::endl;
    exit( - 1 );
   }

   int time       =  atoi( argv[1] );
   int cantidad_N =  atoi( argv[2] );
   int ciclos     =  atoi( argv[3] );

   //-----------------------------------------------------
   // Defino la memoria de los vectores para la velocidad inicial y final

   std:: vector<double>   VI ( cantidad_N );
   std:: vector<double>   VF ( cantidad_N );
   std:: vector<double>   AC ( cantidad_N );

   for (int i = 0; i < cantidad_N; i++)
   {
      VI[i] = 44*0.1; //obtener función en c para calcular numeros aleatorios
      VF[i] = 99*0.1; //
   }

  //------------------------------------------
  // Realizo el cálculo de la aceleración en forma secuencial

  TIEMPO_INI ( HTH_ACEL_SEC )

  for(c=0; c<ciclos; c++)
  {
    for(i=0; i< cantidad_N; i++)
    {
      AC[i] = (VF[i] - VI[i]) / time;
    }
  }

  TIEMPO_FIN ( HTH_ACEL_SEC )

  // ---------------------------------------------
  // Realizo el cálculo de la aceleración con OpenMP

  TIEMPO_INI ( HTH_ACEL_OMP )

  for(c=0; c<ciclos; c++)
  {
    #pragma omp parallel for
    for(i=0; i< cantidad_N; i++)
    {
      AC[i] = (VF[i] - VI[i]) / time;
    }
  }

  TIEMPO_FIN ( HTH_ACEL_OMP )

  //------------------------------------------------

  std::cout<<"Valores Reales  : " <<std::endl;
  std::cout<<"Tiempo total    : " <<TIEMPO_GET( HTH_TOTAL )<<  " [ms]"<<std::endl;
  std::cout<<"Tiempo Acel sec : " <<TIEMPO_GET( HTH_ACEL_SEC)<< " [ms]"<<std::endl;
  std::cout<<"Tiempo Acel Omp : " <<TIEMPO_GET( HTH_ACEL_OMP)<< " [ms]"<<std::endl;
  std::cout<<std::endl;  
}
"""

text_file = open("code_acel.cpp","w")
text_file.write(code)
text_file.close()



##2.2 Compilación de código C (cálculo de aceleración)

In [ ]:
!g++ -o acel -fopenmp code_acel.cpp

#3 Desarrollo

In [ ]:
%env OMP_NUM_THREADS=2

# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidadVehiculos =   2#@param {type: "integer", min:0}
tiempoSegundos    =   5  #@param {type:"slider", min:0, max:100, step:1}
ciclos            =   50  #@param {type:"slider", min:50, max:99, step:1}
# --------------------------------------------
try:
  if cantidadVehiculos < 0:
    raise ValueError("La cantidad de vehículos debe ser mayor a 0");

  if tiempoSegundos == 0:
    raise ValueError("El tiempo debe ser mayor a cero");

#---- Ejecución

  !./acel tiempoSegundos cantidadVehiculos ciclos

#----

except ValueError as valerr:
  print (valerr)
except Exception as e:
  print("Error:",e.args)

env: OMP_NUM_THREADS=2
Valores Reales  : 
Tiempo total    : 1.60593e+09 [ms]
Tiempo Acel sec : 0 [ms]
Tiempo Acel Omp : 0 [ms]



#4 Tabla de pasos

#5 Conclusiones

Crea una seccion paralela y le ponemos la directiva for indicando que es un ciclo for, entonces para cada valor de i, va estar creando un hilo que resuelve una parte del algoritmo

Va a ir tomando tantos valores como el tamaño del vector.

Por defecto, openmp sabe que vamos a iterar por el valor i, porque es el valor que se toma para iterar.

Se crean la cantidad de hilos que le decimos, y mientras dure o haya valores para procesar, los hilos van a seguir vivos.

#6 Bibliografía

[1] Aceleracion: [Referencia](https://www.fisicalab.com/apartado/aceleracion)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Numpy: [Referencia](https://numpy.org/doc/1.16/reference/routines.random.html)

[4] Sliders Parametros: [Referencia](https://colab.research.google.com/notebooks/forms.ipynb)

[5] openMP: [Enlace](https://es.wikipedia.org/wiki/OpenMP)